In [9]:
import numpy as np
import pandas as pd
train=pd.read_csv('C:\\Users\\...\\Downloads\\train.csv', sep=",", header=0, index_col=0)
test=pd.read_csv('C:\\Users\\...\\Downloads\\test.csv', sep=",", header=0, index_col=0)

In [10]:
#to extract the title of the passengers 
train['Name']=train['Name'].str.extract('(, \w+)')
train['Name']=train['Name'].str.extract('(\w+)')

#filling in empty entries and encoding them to be machine learning usable
for x in train.columns:
    train[x]=train[x].fillna(0)

from sklearn import preprocessing
Encoder=preprocessing.LabelEncoder()
columns= ['Name', 'Sex', 'Embarked']    
for x in columns:
    train[x]=Encoder.fit_transform(train[x].astype(str))

#Drop unneeded columns
train.drop('Cabin', axis=1, inplace=True)
train.drop('Ticket', axis=1, inplace=True)

#separating the label from data
Y_train=train['Survived']
X_train=train.drop('Survived',axis=1)


In [11]:
X_train.head()

Pclass  Name  Sex   Age  SibSp  Parch     Fare  Embarked
PassengerId                                                          
1                 3    11    1  22.0      1      0   7.2500         3
2                 1    12    0  38.0      1      0  71.2833         1
3                 3     8    0  26.0      0      0   7.9250         3
4                 1    12    0  35.0      1      0  53.1000         3
5                 3    11    1  35.0      0      0   8.0500         3

In [12]:

#performing grid search to find best hyperparameter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
XGB = XGBClassifier(random_state=0)
parameter_grid = {'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]}

cross_validation = StratifiedKFold(n_splits=3,random_state=0,shuffle=True)

grid_search_XGB = GridSearchCV(XGB,
                           param_grid=parameter_grid,
                           cv=cross_validation,
                            n_jobs=-1,
                            verbose=0)

grid_search_XGB.fit(X_train, Y_train)
print('Best score: {}'.format(grid_search_XGB.best_score_))
print('Best parameters: {}'.format(grid_search_XGB.best_params_))
 

Best score: 0.8249158249158249
Best parameters: {'colsample_bytree': 1.0, 'gamma': 0.5, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 0.8}


In [13]:

for i in np.arange(0,5):
    XGB = XGBClassifier(random_state=i)
    parameter_grid = {'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]}

    cross_validation = StratifiedKFold(n_splits=5,random_state=0,shuffle=True)

    grid_search_XGB = GridSearchCV(XGB,
                           param_grid=parameter_grid,
                           cv=cross_validation,
                            n_jobs=-1,
                            verbose=0)

    grid_search_XGB.fit(X_train, Y_train)
    print('Xgboost Random state:{}'.format(i),'Best score: {}'.format(grid_search_XGB.best_score_))
   


Xgboost Random state:0 Best score: 0.835016835016835
Xgboost Random state:1 Best score: 0.8361391694725028
Xgboost Random state:2 Best score: 0.8383838383838383
Xgboost Random state:3 Best score: 0.8361391694725028
Xgboost Random state:4 Best score: 0.8372615039281706


In [14]:
 parameter_grid = {'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]}

for i in np.arange(0,5):
    for j in np.arange(0,5):
        XGB = XGBClassifier(random_state=i)
        cross_validation = StratifiedKFold(n_splits=3,random_state=j,shuffle=True)
        grid_search_XGB = GridSearchCV(XGB,
                           param_grid=parameter_grid,
                           cv=cross_validation,
                            n_jobs=-1,
                            verbose=0)

        grid_search_XGB.fit(X_train, Y_train)
        print('Xgboost Random state:{}'.format(i),'CV Random state:{}'.format(j),'Best score: {}'.format(grid_search_XGB.best_score_))


Xgboost Random state:0 CV Random state:0 Best score: 0.8249158249158249
Xgboost Random state:0 CV Random state:1 Best score: 0.8305274971941639
Xgboost Random state:0 CV Random state:2 Best score: 0.8395061728395061
Xgboost Random state:0 CV Random state:3 Best score: 0.8417508417508418
Xgboost Random state:0 CV Random state:4 Best score: 0.8439955106621774
Xgboost Random state:1 CV Random state:0 Best score: 0.8271604938271605
Xgboost Random state:1 CV Random state:1 Best score: 0.8249158249158249
Xgboost Random state:1 CV Random state:2 Best score: 0.8406285072951739
Xgboost Random state:1 CV Random state:3 Best score: 0.8395061728395061
Xgboost Random state:1 CV Random state:4 Best score: 0.8428731762065096
Xgboost Random state:2 CV Random state:0 Best score: 0.8271604938271605
Xgboost Random state:2 CV Random state:1 Best score: 0.8226711560044894
Xgboost Random state:2 CV Random state:2 Best score: 0.8395061728395061
Xgboost Random state:2 CV Random state:3 Best score: 0.838383838

In [15]:
print('Best score: {}'.format(grid_search_XGB.best_score_))
print('Best parameters: {}'.format(grid_search_XGB.best_params_))

Best score: 0.8428731762065096
Best parameters: {'colsample_bytree': 0.6, 'gamma': 1.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.8}


In [ ]:
grid_search_XGB.best_estimator_